In [1]:
%matplotlib inline

from functools import partial
from typing import *

import numpy as np
import scipy as sp   # scipy is a popular scientific computing library for Python
import scipy.stats   # statistics package
import matplotlib.pyplot as plt

from ase import Atoms
from ase.calculators.mopac import MOPAC
from ase.io import read, write
from ase.visualize import view

import jax
import jax.numpy as jnp
from jaxlib.xla_extension import DeviceArray

/Users/dehuang/.pyenv/versions/3.9.6/lib/python3.9/site-packages/jax/lib/__init__.py:31: UserWarning: JAX on Mac ARM machines is experimental and minimally tested. Please see https://github.com/google/jax/issues/5501 in the event of problems.
  warnings.warn("JAX on Mac ARM machines is experimental and minimally tested. "


# Protein Folding and AlphaFold

## Goal

1. Introduce the **protein-folding problem**, an adjacent problem to the problem of modeling potential-energy surfaces.
2. Introduce **AlphaFold** (v2.0), a neural network model the solves the protein-folding problem.
3. Discuss ways in which solutions to an adjacent problem can inspire solutions in a related domain.

## Background

A CS introduction to proteins ...

### Why Proteins?

1. "Proteins perform a vast array of functions within organisms, including catalysing metabolic reactions, DNA replication, responding to stimuli, providing structure to cells and organisms, and transporting molecules from one location to another." [https://en.wikipedia.org/wiki/Protein](https://en.wikipedia.org/wiki/Protein)
2. Thus understanding proteins is essential to understanding how biological systems work, which can help us with applications such as drug design.

### Example: Insulin

1. Insulin is a protein that the body uses to regulate blood sugar levels [https://en.wikipedia.org/wiki/Insulin](https://en.wikipedia.org/wiki/Insulin).
2. Deficiency in insulin is linked to diabetes.
3. Scientists/doctors can then create insulin to give to insulin-deficient patients [https://en.wikipedia.org/wiki/Insulin_(medication)](https://en.wikipedia.org/wiki/Insulin_(medication)).

### Insulin: Visualization

Insulin is a protein consisting of two **polymers**:
1. a-chain of insulin (see a, [https://www.rcsb.org/3d-view/1TRZ](https://www.rcsb.org/3d-view/1TRZ))
2. b-chain of insulin (see b, [https://www.rcsb.org/3d-view/1TRZ](https://www.rcsb.org/3d-view/1TRZ))

### What is a Protein?

1. Each **polymer** is a sequence of **monomers**.
2. Each monomer is an **amino acid** **residue**.
3. Each **amino acid** is a molecule consisting of carbon, nitrogen, hydrogen, and a side-chain. There are 20 (21) (proteinogenic) amino acids.
4. The **residue** of an amino acid is the part of the amino acid that remains after it is joined to a polymer.

#### Amino Acid

C(NH2)(R)H-C=O(OH)
![amino_acid_structure.png](amino_acid_structure.png)

In [2]:
ala = read('./aa/Ala.xyz')
print(ala.get_chemical_symbols())
print("C(NH2)(H2)H-C=O(OH)")
view(ala, viewer='x3d')

['H', 'O', 'C', 'C', 'N', 'H', 'O', 'C', 'H', 'H', 'H', 'H', 'H']
C(NH2)(H2)H-C=O(OH)


In [3]:
phe = read('./aa/Phe.xyz')
print(phe.get_chemical_symbols())
print("C6H5-CH2-C(NH2)H-C=O(OH)")
view(phe, viewer='x3d')

['H', 'O', 'C', 'C', 'N', 'H', 'O', 'C', 'H', 'H', 'C', 'H', 'H', 'C', 'C', 'C', 'C', 'C', 'H', 'H', 'H', 'H', 'H']
C6H5-CH2-C(NH2)H-C=O(OH)


#### Two Amino Acids

In [4]:
alaPhe = read('./aa/AlaPhe.xyz')
print(alaPhe.get_chemical_symbols())
view(alaPhe, viewer='x3d')

['H', 'O', 'C', 'C', 'N', 'H', 'O', 'C', 'H', 'H', 'H', 'H', 'H', 'H', 'O', 'C', 'C', 'N', 'H', 'O', 'C', 'H', 'H', 'C', 'H', 'H', 'C', 'C', 'C', 'C', 'C', 'H', 'H', 'H', 'H', 'H']


#### Two Amino Acid Residues, i.e., Two Monomer Polymer

1. We eliminate an OH from one amino acid
2. We eliminate an H from an NH2 on the second amino acid
3. We bind the remaining "residues" together.

In [5]:
alaPhe2 = read('./aa/AlaPhe2.xyz')
print(alaPhe2.get_chemical_symbols())
view(alaPhe2, viewer='x3d')

['C', 'C', 'C', 'O', 'O', 'N', 'C', 'O', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'N', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H']


#### Protein: Amino Acid Residue Polymer

![ProteinogenicAminoAcids.png](ProteinogenicAminoAcids.png)

Note that Wikipedia includes **selenocysteine** as a proteinogenic amino acid.

### Protein Folding

1. There is a principal that "structure determines function", so knowing a protein's structure will help us understand it's function.
2. Protein folding asks: given an amino acid residue sequence, what is the corresponding 3D shape?
3. The reason why this is interesting is because we can easily figure out DNA/RNA sequences, and RNA sequences code for amino acid residue sequences. Consequently, one can hope that we can determine protein functions directly from DNA/RNA sequences.

#### Anfinsen's Dogma or the Thermodynamic Hypothesis

"Anfinsen's dogma, also known as the thermodynamic hypothesis, is a postulate in molecular biology. It states that, at least for a small globular protein in its standard physiological environment, the native structure is determined only by the protein's amino acid sequence."
[https://en.wikipedia.org/wiki/Anfinsen%27s_dogma](https://en.wikipedia.org/wiki/Anfinsen%27s_dogma)

#### An ab-initio approach?

1. We know that amino acid residues are molecules that should satisfy the postulates of quantum mechanics.
2. We can perform geometry optimization on these chains.
3. But there chains are too large and neglects the presence of other molecules.

In [6]:
alaPhe2 = read('./aa/AlaPhe2.xyz')
alaPhe2.set_calculator(MOPAC(label=f"tmp/foobar", task=f"PM7 UHF GRAD DISP"))
print('Potential energy', alaPhe2.get_potential_energy())
alaPhe2 = MOPAC.read_atoms('tmp/foobar')
view(alaPhe2, viewer='x3d')

Potential energy -2949.16689
HERE <ase.calculators.mopac.MOPAC object at 0x13f447d60>


## AlphaFold

Alphafold is neural network designed by Google DeepMind that solves the protein folding problem
1. Github: [https://github.com/deepmind/alphafold](https://github.com/deepmind/alphafold)
2. Paper: [https://www.nature.com/articles/s41586-021-03819-2](https://www.nature.com/articles/s41586-021-03819-2)
3. Supplementary material: [https://static-content.springer.com/esm/art%3A10.1038%2Fs41586-021-03819-2/MediaObjects/41586_2021_3819_MOESM1_ESM.pdf](https://static-content.springer.com/esm/art%3A10.1038%2Fs41586-021-03819-2/MediaObjects/41586_2021_3819_MOESM1_ESM.pdf)

### Abstract

1. **AlphaFold solves the protein-folding problem**: "Here we provide the first computational method that can regularly predict protein structures with atomic accuracy even in cases in which no similar structure is known."
2. **The model works on CASP14 and is the winner**: "We validated an entirely redesigned version of our neural network-based model, AlphaFold, in the challenging 14th Critical Assessment of protein Structure Prediction (CASP14) [15], demonstrating accuracy competitive with experimental structures in a majority of cases and greatly outperforming other methods."
3. **Novel deep-learning algorithms and representations developed**: "Underpinning the latest version of AlphaFold is a novel machine learning approach that incorporates physical and biological knowledge about protein structure, leveraging multi-sequence alignments, into the design of the deep learning algorithm."

### Introduction 

![media/alphafold-fig1.png](alphafold-fig1.png)

1. a: AlphaFold wins the CASP14 competition.
2. b: AlphaFold's prediction of [https://www.rcsb.org/3d-view/6Y4F](https://www.rcsb.org/3d-view/6Y4F), c: AlphaFold's prediction of [https://www.rcsb.org/structure/6yj1](https://www.rcsb.org/structure/6yj1), d: AlphaFold's prediction of [https://www.rcsb.org/structure/6vr4](https://www.rcsb.org/structure/6vr4). Protein distances can be measured with RMSD (root mean squared distance) and TM (template-modeling) scores.
3. e: AlphaFold's neural network architecture and representation of data, including the usage of MSA (multiple sequence alignment), a pairing representation of amino-acid residues, the Evoformer block, a structure module, and the technique of recycling.

### Conclusion 

1. AlphaFold still can really only predict in data proteins: "In general, AlphaFold is trained to produce the protein structure most likely to appear as part of a PDB structure. For example, in cases in which a particular stochiometry, ligand or ion is predictable from the sequence alone, AlphaFold is likely to produce a structure that respects those constraints implicitly."
2. AlphaFold's significance is that it's enabling us to take advantage of genomic data: "... we hope to accelerate the advancement of structural bioinformatics that can keep pace with the genomics revolution. We hope that AlphaFold—and computational approaches that apply its techniques for other biophysical problems—will become essential tools of modern biology."

### Experiments 

1. We would like to quantify how good the predicted structures are to reference structures.
2. Consequently, we need a way to measure the "distance" between two proteins.

#### Experiment 3a: How good are the protein structures?

#### Question: how do we measure distance?

The **$C\alpha$ carbon** is the carbon bound to the OH group. [https://en.wikipedia.org/wiki/Alpha_and_beta_carbon](https://en.wikipedia.org/wiki/Alpha_and_beta_carbon)
<img src="media/amino_acid_structure.png" alt="amino_acid_structure.png" width="200"/>

#### Example: Ala-Phe

<img src="media/ala-phe.png" alt="ala-phe.png" width="200"/>

#### RMSD

RMSD means **root mean squared distance** 
$$
RMSD(p, q) = \sqrt{\frac{1}{n} \sum_{i=1}^n \lVert p_i - q_i \rVert}
$$
is the square root of the average distance between the residue positions (the position of the $C\alpha$ carbon) of an input p and a reference q.

#### Experiment 3a Results

1. The alignment of the input p and the reference q can have error, so the .95 means we select the top 95% of residues that have the best alignment.
2. The bars are **confidence intervals** assuming a poisson distribution.

<img src="media/alphafold-expa.png" alt="alphafold-expa.png" width="200"/>

#### Experiment 3b: How good are the angles?

1. "**Local Distance Difference Test (lDDT)** is a superposition-free score that evaluates local distance differences of all atoms in a model, including validation of stereochemical plausibility." [https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3799472/](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3799472/) Thus 1DDT-C$\alpha$ score measures the local distance between all $\alpha$-carbons of residues in a way that accounts for stereochemistry, i.e., relative positions of atoms (e.g., by rotation).
2. A **rotamer** is a conformer that is generated by rotation. "A rotamer is classified as correct if the predicted torsion angle is within 40°. Each point aggregates a range of lDDT-Cα, with a bin size of 2 units above 70 lDDT-Cα and 5 units otherwise. Points correspond to the mean accuracy; error bars are 95% confidence intervals (Student t-test) of the mean on a per-residue basis." 

#### Experiment 3b: Results

Taking into account experimental measurements. "Filtered to structures with any observed side chains and resolution better than 2.5 Å (n = 5,317 protein chains); side chains were further filtered to B-factor <30 Å2".

<img src="media/alphafold-expb.png" alt="alphafold-expb.png" width="400"/>

#### Experiment 3c: Neural Network Confidence?

1. **Predicted local-distance difference test (pLDDT)** so how does the neural network predict this number?
2. **Resolved region** is the part of the protein that we have experimental confirmation of its structure.
3. Want this to be as close to slope 1 as possible.
<img src="media/alphafold-expc.png" alt="alphafold-expc.png" width="400"/>

#### Experiment 3d: Neural Network Confidence?

1. **pTM (predicted template-modeling)** so how does the neural network predict this number?
2. **Resolved region** is the part of the protein that we have experimental confirmation of its structure.
3. Want this to be as close to slope 1 as possible.

<img src="media/alphafold-expd.png" alt="alphafold-expd.png" width="400"/>

### Methods

How does it work?

<img src="media/alphafold-methods.png" alt="methods.png" width="300"/>